In [1]:
with open('../data/wine_links_0_to_6.txt', 'r') as a, open('../data/wine_links_6_to_7.txt', 'r') as b, open('../data/wine_links_7_to_8.txt', 'r') as c, open('../data/wine_links_8_to_9.txt', 'r') as d, open('../data/wine_links_9_to_11.txt', 'r') as e, open('../data/wine_links_11_to_12.txt', 'r') as f, open('../data/wine_links_12_to_15.txt', 'r') as g, open('../data/wine_links_15_to_16.txt', 'r') as h, open('../data/wine_links_16_to_20.txt', 'r') as i, open('../data/wine_links_20_to_21.txt', 'r') as j, open('../data/wine_links_21_to_30.txt', 'r') as k, open('../data/wine_links_30_to_60.txt', 'r') as l, open('../data/wine_links_60_to_500plus.txt', 'r') as m:
    
    links = a.readlines()+b.readlines()+c.readlines()+d.readlines()+e.readlines()+f.readlines()+g.readlines()+h.readlines()+i.readlines()+j.readlines()+k.readlines()+l.readlines()+m.readlines()

In [2]:
len(links)

12588

In [3]:
links = [link.replace("\n", "") for link in links]

In [4]:
links = [link for link in links if "https://www.vivino.com" in link]

In [5]:
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [7]:
PATH=ChromeDriverManager().install()

driver = webdriver.Chrome(PATH,options = opciones)

C:\Users\eloyl\AppData\Local\Temp\ipykernel_11224\2256630989.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH,options = opciones)


In [8]:
complete_data = []

In [9]:
def data_from_links(url):
    try:
        # start driver 
        driver=webdriver.Chrome(PATH, options= opciones)
        driver.get(url)

        time.sleep(4)
    
        #cookies banner selection
        btn_cookie = driver.find_element(By.XPATH, '//*[@id="cookie-notice-container"]/div/button')
        #cookie banner click
        btn_cookie.click()
    
        time.sleep(3)
        
        #end page
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        
        #data
        top_web = driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div[2]')
        wine = [i.text for i in top_web.find_elements(By.TAG_NAME, "a")]

        #save data to variables
        
        bodega = driver.find_element(By.CLASS_NAME, "row.header.breadCrumbs").text.split("\n")[0]
        
        nombre = driver.find_element(By.CLASS_NAME, "row.header.breadCrumbs").text.split("\n")[1]
        
        valoracion = [i for i in wine if "valoraciones" in i][0].split("\n")[0]
        
        num_valoraciones = [i for i in wine if "valoraciones" in i][0].split("\n")[1].split(" ")[0]
        
        precio = [i for i in driver.find_element(By.CLASS_NAME, "vintageSelection").text.split("\n") if "€" in i][0][1:]
        if len(precio) > 6:
            precio = None
         
        try:
            año = url.split("?")[1].split("&")[0][-4:]
        except:
            año = None


        zona = [i for i in [i.text for i in driver.find_elements(By.CSS_SELECTOR, "tr")] if "Región" in i][0].split("\n")[1]
        
        tipo = [i for i in wine if "Vino" in i][0]
        
        uva= [i for i in [i.text for i in driver.find_elements(By.CSS_SELECTOR, "tr")] if "Uvas" in i][0].split("\n")[1]
        #save data to a dictionary and export
        complete_data = {
    
        'winery': bodega,
        'wine': nombre,
        'year': año,
        'rating': valoracion,
        'num_review': num_valoraciones,
        'region': zona,
        'price': precio,
        'type': tipo,
        'grapes' : uva
    
        } 
    
        return complete_data
    except:
        return url

In [10]:
from joblib import Parallel, delayed

import pandas as pd

from tqdm.notebook import tqdm 

In [13]:
len(links)

12213

In [12]:
links = list(set(links))

In [ ]:
len(links)/2

In [ ]:
links1 = links[:1000]

In [ ]:
%%time
data1 = []

data1 = [Parallel(n_jobs=10, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links1))]

In [ ]:
data1 = data1[0]

In [ ]:
data1

In [ ]:
len(data1)

In [ ]:
missing_data1 = []
for i in data1:
    if len(i) > 9:
        missing_data1.append(i)

In [ ]:
missing_data1

In [ ]:
for i in data1:
    if len(i) > 9:
        data1.remove(i)

In [ ]:
for i in data1:
    if len(i) > 9:
        print(i)

In [ ]:
for i in data1:
    if len(i) > 9:
        data1.remove(i)

In [ ]:
df_wines1 = pd.DataFrame(data1)

In [ ]:
df_wines1.to_csv('../data/allwines1.csv',index=False, header=True)

In [ ]:
links2 = links[1000:1300]

In [ ]:
%%time
data2 = []

data2 = [Parallel(n_jobs=10, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links2))]

In [ ]:
data2 = data2[0]

In [ ]:
for i in data2:
    if len(i) > 9:
        print(i)

In [ ]:
for i in data2:
    if len(i) > 9:
        missing_data1.append(i)

In [ ]:
for i in data2:
    if len(i) > 9:
        data2.remove(i)

In [ ]:
df_wines2 = pd.DataFrame(data2)

In [ ]:
df_wines2.to_csv('../data/allwines2.csv',index=False, header=True)

In [ ]:
missing_data1

In [ ]:
links3 = links[1300:2300]

In [ ]:
%%time
data3 = []

data3 = [Parallel(n_jobs=10, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links3))]

In [ ]:
data3 = data3[0]

In [ ]:
for i in data3:
    if len(i) > 9:
        print(i)

In [ ]:
for i in data3:
    if len(i) > 9:
        missing_data1.append(i)

In [ ]:
len(missing_data1)

In [ ]:
for i in data3:
    if len(i) > 9:
        data3.remove(i)

In [ ]:
df_wines3 = pd.DataFrame(data3)

In [ ]:
df_wines3.to_csv('../data/allwines3.csv',index=False, header=True)

In [ ]:
with open("../data/missing_values.txt", "w") as m:
    for line in missing_data1:
        m.write(line + "\n")

In [ ]:
links4 = links[2300:3300]

In [ ]:
%%time
data4 = []

data4 = [Parallel(n_jobs=10, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links4))]

In [ ]:
data4 = data4[0]

In [ ]:
for i in data4:
    if len(i) > 9:
        print(i)

In [ ]:
for i in data4:
    if len(i) > 9:
        missing_data1.append(i)

In [ ]:
for i in data4:
    if len(i) > 9:
        data4.remove(i)

In [ ]:
df_wines4 = pd.DataFrame(data4)

In [ ]:
df_wines4.to_csv('../data/allwines4.csv',index=False, header=True)

In [ ]:
len(missing_data1)

In [ ]:
links5 = links[3300:3800]

In [ ]:
%%time
data5 = []

data5 = [Parallel(n_jobs=10, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links5))]

In [ ]:
data5 = data5[0]

In [ ]:
for i in data5:
    if len(i) > 9:
        print(i)

In [ ]:
for i in data5:
    if len(i) > 9:
        missing_data1.append(i)

In [ ]:
for i in data5:
    if len(i) > 9:
        data5.remove(i)

In [ ]:
df_wines5 = pd.DataFrame(data5)

In [ ]:
df_wines5.to_csv('../data/allwines5.csv',index=False, header=True)

In [ ]:
with open("../data/missing_values2.txt", "w") as m:
    for line in missing_data1:
        m.write(line + "\n")

In [ ]:
len(missing_data1)

In [ ]:
len(links)

In [7]:
links6 = links[3800:5800]

In [15]:
%%time
data6 = []

data6 = [Parallel(n_jobs=8, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links6))]

  0%|          | 0/2000 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  7.8min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed: 18.4min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed: 33.6min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed: 51.6min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed: 73.6min


CPU times: total: 5.33 s
Wall time: 1h 21min 58s


[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed: 82.0min finished


In [16]:
data6 = data6[0]

In [26]:
for i in data6:
    if len(i) > 9:
        print(i)

In [18]:
missing_data = []

In [19]:
for i in data6:
    if len(i) > 9:
        missing_data.append(i)

In [20]:
missing_data

['https://www.vivino.com/ES/es/algueira-serradelo/w/6664393?year=2015&price_id=19941387',
 'https://www.vivino.com/ES/es/perinet-likka/w/9132860?year=2018&price_id=25664639',
 'https://www.vivino.com/ES/es/garriguella-emporda-puntils/w/1878834?year=2019&price_id=23154377',
 'https://www.vivino.com/toplists/top-25-spanish-pedro-ximenez-wines-spain-right-now-spain',
 'https://www.vivino.com/ES/es/alta-alella-bruant/w/4023543?year=2018&price_id=23260601',
 'https://www.vivino.com/ES/es/algueira-anadelia-ribeira-sacra/w/5916679?year=2015&price_id=18216567',
 'https://www.vivino.com/ES/es/fuentenarro-esenzias-18-meses-en-barrica-de-roble-frances/w/7118794?year=2018&price_id=26239454',
 'https://www.vivino.com/ES/es/bodega-can-verdura-supernova-mantonegro-negre/w/5812908?year=2020&price_id=27224672',
 'https://www.vivino.com/ES/es/jose-pariente-finca-las-comas-verdejo/w/7273309?year=2016&price_id=19360246',
 'https://www.vivino.com/ES/es/real-divisa-marques-de-legarda-crianza/w/1396841?year=

In [25]:
for i in data6:
    if len(i) > 9:
        data6.remove(i)

In [27]:
df_wines6 = pd.DataFrame(data6)

In [28]:
df_wines6.to_csv('../data/allwines6.csv',index=False, header=True)

In [29]:
with open("../data/missing_values3.txt", "w") as m:
    for line in missing_data:
        m.write(line + "\n")

In [14]:
links7 = links[5800:6800]

In [15]:
len(links7)

1000

In [16]:
%%time
data7 = []

data7 = [Parallel(n_jobs=12, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links7))]

  0%|          | 0/1000 [00:00<?, ?it/s]

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.3min
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:  5.7min
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed: 13.7min
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed: 24.0min


CPU times: total: 3.7 s
Wall time: 30min 38s


[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed: 30.6min finished


In [17]:
data7 = data7[0]

In [27]:
for i in data7:
    if len(i) > 9:
        print(i)

In [20]:
missing_data = []

In [21]:
for i in data7:
    if len(i) > 9:
        missing_data.append(i)

In [26]:
for i in data7:
    if len(i) > 9:
        data7.remove(i)

In [28]:
df_wines7 = pd.DataFrame(data7)

In [29]:
df_wines7.to_csv('../data/allwines7.csv',index=False, header=True)

In [30]:
with open("../data/missing_values4.txt", "w") as m:
    for line in missing_data:
        m.write(line + "\n")

In [31]:
links8 = links[6800:7800]

In [32]:
%%time
data8 = []

data8 = [Parallel(n_jobs=12, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links8))]

  0%|          | 0/1000 [00:00<?, ?it/s]

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.3min
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:  6.1min
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed: 13.7min
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed: 24.2min


CPU times: total: 4.05 s
Wall time: 31min 3s


[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed: 31.1min finished


In [33]:
data8 = data8[0]

In [37]:
for i in data8:
    if len(i) > 9:
        print(i)

In [35]:
for i in data8:
    if len(i) > 9:
        missing_data.append(i)

In [36]:
for i in data8:
    if len(i) > 9:
        data8.remove(i)

In [38]:
df_wines8 = pd.DataFrame(data8)

In [39]:
df_wines8.to_csv('../data/allwines8.csv',index=False, header=True)

In [40]:
with open("../data/missing_values4.txt", "w") as m:
    for line in missing_data:
        m.write(line + "\n")

In [42]:
links9 = links[7800:8800]

In [43]:
%%time
data9 = []

data9 = [Parallel(n_jobs=12, verbose=True)(delayed(data_from_links)(url) for url in tqdm(links9))]

  0%|          | 0/1000 [00:00<?, ?it/s]

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.3min
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:  6.4min
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed: 14.2min
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed: 25.1min


CPU times: total: 3.55 s
Wall time: 32min 5s


[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed: 32.1min finished


In [44]:
data9 =data9[0]

In [52]:
for i in data9:
    if len(i) > 9:
        print(i)

In [46]:
for i in data9:
    if len(i) > 9:
        missing_data.append(i)

In [51]:
for i in data9:
    if len(i) > 9:
        data9.remove(i)

In [53]:
df_wines9 = pd.DataFrame(data9)

In [54]:
df_wines9.to_csv('../data/allwines9.csv',index=False, header=True)

In [55]:
with open("../data/missing_values4.txt", "w") as m:
    for line in missing_data:
        m.write(line + "\n")